## Imports

In [ ]:
# debug
import sys

print(sys.executable)
print(sys.version_info)

In [ ]:
# main imports
import holoviews as hv
import panel as pn
import numpy as np
import pandas as pd
import xarray as xr

pn.__version__

hv.extension("bokeh")

In [ ]:
# Set some defaults for the holoviews plots
DEFAULTS = {
    "show_title": True, 
    "active_tools": ["box_zoom"],
    "height": 300,
    "width": 600
}

hv.opts.defaults(
    hv.opts.Curve(**DEFAULTS),
    hv.opts.Image(**DEFAULTS, cmap="jet"),
    hv.opts.Points(**DEFAULTS),
)

## Open a dataset

Thalassa supports the output of several solvers (e.g. Schism, ADCIRC). 
In order to do so, it converts the output of these files to a common "schema".
This process is called "normalization".


<div class="alert alert-info" role="alert">
    "normalization" == Renaming of dimensions and variables
</div>

The most convenient way to apply this normalization process is to use the `api.open_dataset()` function which is a wrapper around `xr.open_dataset()`.

In [ ]:
from thalassa import api

api.open_dataset?

In [ ]:
filename = "../tests/data/fort.63.nc"
ds = api.open_dataset(filename)
ds

## Main API

`thalassa` supports several types of graphs, including:
    
- A visualization of the variables at various timestamps and/or layers.
- A visualization of the mesh
- Extraction of timeseries from specific points

The following cell shows the basic usage:

In [ ]:
#variable, layer, timestamp = "salt", 40, 
#variable, layer, timestamp = "depth", None, None
variable, layer, timestamp = "zeta", None, ds.time.values[4]

# The trimesh is the most basic object. This is what you need to create all the others graphs
# It is on this object that you specify the timestamp and/or the layer.
trimesh = api.create_trimesh(ds.sel(time=timestamp), variable=variable)

# The nodes of the mesh (without triangles, just the points!)
nodes = api.get_nodes(trimesh)

# The wireframe is the representation of the mesh
wireframe = api.get_wireframe(trimesh)

# The tiles is using the tiling service from Open Street maps
tiles =  api.get_tiles() 

# The raster object is the basic Map that visualizes the variable. 
# You can specify things like the colorbar limits and/or the extents
#raster = api.get_raster(trimesh, clim_min=0, clim_max=15)
raster = api.get_raster(trimesh)

# The pointer/tap timeseries extract the timeseries of a specific node from the xr.Dataset and visualize it.
pointer_dmap = api.get_pointer_timeseries(ds=ds, variable=variable, source_raster=raster)
tap_dmap = api.get_tap_timeseries(ds=ds, variable=variable, source_raster=raster)

### Let's visualize!

After you render the layout, move the mouse over the map and click on it. This will fill in the `pointer_dmap` and the `tap_dmap`.

In [ ]:
raster_layout = tiles * raster.opts(width=600, cmap="viridis") 
raster_layout + tap_dmap

In [ ]:
tiles * nodes.opts(width=600)

In [ ]:
tiles * wireframe.opts(width=600)

In [ ]:
tiles * raster * wireframe * nodes.opts(width=600, size=2)